## Pytorch-Training
This file trains a PyTorchModel for use in Gradient ascent fooling. 

this was necessary as there are no similiar libraries for tensorflow at the moment.

This Model is not build from scratch, but starts with the famous alex-net. 

This is just for documentation, please just use the model provided with the code. 

In [ ]:
from __future__ import print_function
import zipfile
import os

import torchvision.transforms as transforms
from torchvision import datasets, transforms

import PIL
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
import torch.optim as optim
import datetime

import shutil
import time

import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torchvision.datasets as datasets


IMG_SIZE = 64
NUM_CLASSES = 43

In [2]:
# Prepare dataset
def prepare_dataset(folder):
    train_zip = folder + '/train_images.zip'
    test_zip = folder + '/test_images.zip'
    if not os.path.exists(train_zip) or not os.path.exists(test_zip):
        raise(RuntimeError("Could not find " + train_zip + " and " + test_zip))

    # extract train_data.zip to train_data
    train_folder = folder + '/train_images'

    if not os.path.isdir(train_folder):
        print(train_folder + ' not found, extracting ' + train_zip)
        zip_ref = zipfile.ZipFile(train_zip, 'r')
        zip_ref.extractall(folder)
        zip_ref.close()

    # extract test_data.zip to test_data
    test_folder = folder + '/test_images'
    if not os.path.isdir(test_folder):
        print(test_folder + ' not found, extracting ' + test_zip)
        zip_ref = zipfile.ZipFile(test_zip, 'r')
        zip_ref.extractall(folder)
        zip_ref.close()

    # make validation_data by using images 00000*, 00001* and 00002* in each class
    val_folder = folder + '/val_images'
    if not os.path.isdir(val_folder):
        print(val_folder + ' not found, making a validation set')
        os.mkdir(val_folder)
        for dirs in os.listdir(train_folder):
            if dirs.startswith('000'):
                os.mkdir(val_folder + '/' + dirs)
                for f in os.listdir(train_folder + '/' + dirs):
                    if f.startswith('00000') or f.startswith('00001') or f.startswith('00002'):
                        # move file to validation folder
                        os.rename(train_folder + '/' + dirs + '/' + f, val_folder + '/' + dirs + '/' + f)


In [3]:
def train(train_loader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    # switch to train mode
    model.train()

    end = time.time()
    for i, (input, target) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        if torch.cuda.is_available():
            target = target.cuda()
        else:
            target = target.cpu()

        input_var = torch.autograd.Variable(input)
        target_var = torch.autograd.Variable(target)

        # compute output
        output = model(input_var)
        loss = criterion(output, target_var)

        # measure accuracy and record loss
        prec1, prec5 = accuracy(output.data, target, topk=(1, 5))
        losses.update(loss.item(), input.size(0))
        top1.update(prec1.item(), input.size(0))
        top5.update(prec5.item(), input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % 10 == 0:

            out_train = ('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec@1 {top1.val:.3f} ({top1.avg:.3f})\t'
                  'Prec@5 {top5.val:.3f} ({top5.avg:.3f})').format(
                   epoch, i, len(train_loader), batch_time=batch_time,
                   data_time=data_time, loss=losses, top1=top1, top5=top5)

            with open("######Progress-training.txt", "a") as myfile:
                myfile.write(str(datetime.datetime.now()) + " -- " + out_train + "\n")
            print(out_train)

def validate(val_loader, model, criterion):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    for i, (input, target) in enumerate(val_loader):
        if torch.cuda.is_available():
            target = target.cuda()
        else:
            target = target.cpu()

        input_var = torch.autograd.Variable(input, volatile=True)
        target_var = torch.autograd.Variable(target, volatile=True)

        # compute output
        output = model(input_var)
        loss = criterion(output, target_var)

        # measure accuracy and record loss
        prec1, prec5 = accuracy(output.data, target, topk=(1, 5))
        losses.update(loss.item(), input.size(0))
        top1.update(prec1.item(), input.size(0))
        top5.update(prec5.item(), input.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % 10 == 0:
            out_val = ('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec@1 {top1.val:.3f} ({top1.avg:.3f})\t'
                  'Prec@5 {top5.val:.3f} ({top5.avg:.3f})').format(
                   i, len(val_loader), batch_time=batch_time, loss=losses,
                   top1=top1, top5=top5)
            with open("######Progress-validation.txt", "a") as myfile:
                myfile.write(str(datetime.datetime.now()) + " -- " + out_val + "\n")
            print(out_val)

    out_val_final = (' * Prec@1 {top1.avg:.3f} Prec@5 {top5.avg:.3f}'
          .format(top1=top1, top5=top5))
    with open("######Progress-validation-final.txt", "a") as myfile:
        myfile.write(str(datetime.datetime.now()) + " -- " + out_val_final + "\n")
    print(out_val)
    return top1.avg




def save_checkpoint(state, is_best, filename='checkpoint.pth'):
    #torch.save(state, filename)
    torch.save(model.state_dict(), filename)
    if is_best:
        shutil.copyfile(filename, 'model_best.pth')




def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
#   lr = 0.1 * (0.1 ** (epoch // 30)) # first value is learning rate (1st trial)
#    lr = 0.01 * (0.1 ** (epoch // 30)) # first value is learning rate (2nd trial)
#    lr = 0.001 * (0.1 ** (epoch // 30)) # first value is learning rate (3rd trial)
    lr = 0.01 * (0.1 ** (epoch // 30)) # first value is learning rate (4th trial)

    for param_group in optimizer.param_groups:
        param_group['lr'] = lr




def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
prepare_dataset('data')
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

traindir = 'data/train_images'
train_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder(traindir, transforms.Compose(
        [
            transforms.Resize((IMG_SIZE, IMG_SIZE)),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            normalize,
        ]
    )), batch_size=64, shuffle=True, num_workers=1)

valdir = 'data/val_images'
val_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder(valdir, transforms.Compose(
        [
            transforms.Resize((IMG_SIZE, IMG_SIZE)),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            normalize,
        ]
    )), batch_size=64, shuffle=True, num_workers=1)

from model import AlexNet

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = AlexNet().to(device)
use_sgd_optimizer = False

if use_sgd_optimizer == True:
    if torch.cuda.is_available():
        criterion = nn.CrossEntropyLoss().cuda()
    else:
        criterion = nn.CrossEntropyLoss()

    optimizer = torch.optim.SGD(model.parameters(),
                                0.01, # learning rate (2nd trial)
                                momentum=0.9,
                                weight_decay=1e-6) # decay (2nd trial)
else:
    optimizer = optim.Adam(model.parameters(), lr=0.01)

#####################
epochs = 51 # trains for 50 epochs
best_prec1 = 0
#####################
for epoch in range(1, epochs):
    progress_epochs = "\n============================\nStart training epoch: " + str(epoch) + "/" + str(epochs-1) + "\n@ " + str(datetime.datetime.now()) + "\n============================"
    with open("######Progress-epochs.txt", "a") as myfile:
        myfile.write(progress_epochs + "\n")

    print(progress_epochs)    


    adjust_learning_rate(optimizer, epoch)

    # train for one epoch
    train(train_loader, model, criterion, optimizer, epoch)

    # evaluate on validation set
    prec1 = validate(val_loader, model, criterion)

    # remember best prec@1 and save checkpoint
    is_best = prec1 > best_prec1
    best_prec1 = max(prec1, best_prec1)
    save_checkpoint({
        'epoch': epoch + 1,
        'arch': "alexnet",
        'state_dict': model.state_dict(),
        'best_prec1': best_prec1,
    }, is_best)

    progress_epochs = "\n=============================\nFinished training epoch: " + str(epoch) + "/" + str(epochs-1) + "\n@ " + str(datetime.datetime.now()) + "\n============================="

    with open("######Progress-epochs.txt", "a") as myfile:
        myfile.write(progress_epochs + "\n")
    print(progress_epochs)  


Start training epoch: 1/50
@ 2018-12-21 05:47:54.137344
Epoch: [1][0/354]	Time 3.239 (3.239)	Data 0.182 (0.182)	Loss 3.7597 (3.7597)	Prec@1 0.000 (0.000)	Prec@5 13.000 (13.000)
Epoch: [1][10/354]	Time 2.345 (2.368)	Data 0.003 (0.021)	Loss 3.7405 (3.7517)	Prec@1 3.000 (3.818)	Prec@5 25.000 (20.636)
Epoch: [1][20/354]	Time 2.284 (2.363)	Data 0.005 (0.013)	Loss 3.7148 (3.7386)	Prec@1 4.000 (4.810)	Prec@5 22.000 (23.952)
Epoch: [1][30/354]	Time 2.310 (2.358)	Data 0.002 (0.010)	Loss 3.6753 (3.7217)	Prec@1 3.000 (4.710)	Prec@5 21.000 (25.323)
Epoch: [1][40/354]	Time 2.271 (2.344)	Data 0.002 (0.008)	Loss 3.5673 (3.6931)	Prec@1 1.000 (5.244)	Prec@5 25.000 (26.146)
Epoch: [1][50/354]	Time 2.295 (2.333)	Data 0.003 (0.007)	Loss 3.5155 (3.6469)	Prec@1 1.000 (5.471)	Prec@5 22.000 (26.333)
Epoch: [1][60/354]	Time 2.273 (2.326)	Data 0.002 (0.006)	Loss 3.4240 (3.6124)	Prec@1 8.000 (5.508)	Prec@5 29.000 (26.410)
Epoch: [1][70/354]	Time 2.223 (2.333)	Data 0.002 (0.006)	Loss 3.4641 (3.5922)	Prec@1 8.000